In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import libraries and data

In [2]:
!pip install bert-score
!pip install rouge_score
!pip install pycocoevalcap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing in

In [3]:
# Import libraries
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import json
from torch.nn.utils.rnn import pad_sequence
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from bert_score import score as bert_score
from transformers import AutoModel, AutoTokenizer
from pycocoevalcap.cider.cider import Cider

2025-05-01 07:47:35.118777: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746085655.354473      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746085655.427425      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# Load AITeamVN/Vietnamese_Embedding
embed_model = SentenceTransformer("AITeamVN/Vietnamese_Embedding")

# Load annotation
vocab_dir = "/kaggle/input/capydata-ic/data/vocab"
with open(os.path.join(vocab_dir, "vocab.json"), "r", encoding="utf-8") as f:
    vocab = json.load(f)
    vocab = {k: int(v) for k, v in vocab.items()}
with open(os.path.join(vocab_dir, "idx_to_word.json"), "r", encoding="utf-8") as f:
    idx_to_word = json.load(f)
    idx_to_word = {int(k): v for k, v in idx_to_word.items()}
    inv_vocab = idx_to_word

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.38k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/708 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

## Define functions/classes

In [5]:
# Define functions/classes
class ImageCaptionDataset(Dataset):
    def __init__(self, image_dir, captions_file, vocab, transform=None):
        self.image_dir = image_dir
        self.vocab = vocab
        self.transform = transform
        self.data = []
        with open(captions_file, 'r', encoding='utf-8') as f:
            for line in f:
                img_name, caption = line.strip().split('\t')
                tokens = caption.strip().split()
                self.data.append((img_name, tokens))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name, tokens = self.data[idx]
        image_path = os.path.join(self.image_dir, img_name)
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        caption = [self.vocab['<s>']] + [self.vocab.get(token, self.vocab['<unk>']) for token in tokens] + [self.vocab['</s>']]
        return image, torch.tensor(caption), img_name, tokens

def collate_fn(batch):
    images, captions, img_names, raw_tokens = zip(*batch)
    images = torch.stack(images)
    captions = pad_sequence(captions, batch_first=True, padding_value=vocab['<pad>'])
    return images, captions, img_names, raw_tokens

In [6]:
# Encoder CNN
class EncoderCNN(nn.Module):
    def __init__(self, encoded_size=256):
        super().__init__()
        resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        # Unfreeze last 2 layers for fine-tuning
        for param in resnet.parameters():
            param.requires_grad = False
            
        modules = list(resnet.children())
        # Unfreeze parameters in the last two layers
        for layer in modules[-2:]:
            for param in layer.parameters():
                param.requires_grad = True
                
        # Remove the last FC layer
        self.resnet = nn.Sequential(*modules[:-2])
        self.fc = nn.Linear(2048, encoded_size)
        self.bn = nn.BatchNorm1d(encoded_size)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))

    def forward(self, images):
        features = self.resnet(images)
        features = self.adaptive_pool(features)
        features = features.view(features.size(0), -1)
        features = self.fc(features)
        features = self.bn(features)
        return features

# Attention mechanism
class Attention(nn.Module):
    def __init__(self, encoder_dim, decoder_dim, attention_dim):
        super().__init__()
        self.encoder_att = nn.Linear(encoder_dim, attention_dim)
        self.decoder_att = nn.Linear(decoder_dim, attention_dim)
        self.full_att = nn.Linear(attention_dim, 1)

    def forward(self, encoder_out, decoder_hidden):
        att1 = self.encoder_att(encoder_out)
        att2 = self.decoder_att(decoder_hidden).unsqueeze(1)
        att = self.full_att(torch.tanh(att1 + att2)).squeeze(2)
        alpha = torch.softmax(att, dim=1)
        context = (encoder_out * alpha.unsqueeze(2)).sum(dim=1)
        return context, alpha

# Decoder RNN
class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, encoder_dim=256, attention_dim=256):
        super().__init__()
        self.attention = Attention(encoder_dim, hidden_size, attention_dim)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTMCell(embed_size + encoder_dim, hidden_size)
        self.fc = nn.Linear(hidden_size, vocab_size)
        self.dropoutOthers = nn.Dropout(0.6)
        self.dropoutLSTM = nn.Dropout(0.6)
        self.hidden_size = hidden_size

    def forward(self, encoder_out, captions, teacher_forcing_ratio=0.5):
        batch_size = encoder_out.size(0)
        vocab_size = self.fc.out_features
        embeddings = self.embedding(captions)
        embeddings = self.dropoutLSTM(embeddings)
        h, c = torch.zeros(batch_size, self.hidden_size).to(encoder_out.device), torch.zeros(batch_size, self.hidden_size).to(encoder_out.device)
        outputs = torch.zeros(batch_size, captions.size(1), vocab_size).to(encoder_out.device)
        
        for t in range(captions.size(1)):
            context, _ = self.attention(encoder_out.unsqueeze(1), h)
            input_lstm = torch.cat([embeddings[:, t], context], dim=1)
            input_lstm = self.dropoutLSTM(input_lstm)
            h, c = self.lstm(input_lstm, (h, c))
            h = self.dropoutOthers(h)
            outputs[:, t] = self.fc(h)
        return outputs

# Combined Model
class ImageCaptionModel(nn.Module):
    def __init__(self, encoded_size=256, embed_size=256, hidden_size=512, vocab_size=len(vocab), attention_dim=256):
        super().__init__()
        self.encoder = EncoderCNN(encoded_size=encoded_size)
        self.decoder = DecoderRNN(embed_size=embed_size, hidden_size=hidden_size, vocab_size=vocab_size, 
                                 encoder_dim=encoded_size, attention_dim=attention_dim)
        # Load AITeamVN/Vietnamese_Embedding embeddings
        pretrained_embeds = self.get_vietnamese_embeddings(vocab, embed_size)
        self.decoder.embedding.weight.data = pretrained_embeds
        self.decoder.embedding.weight.requires_grad = True

    def get_vietnamese_embeddings(self, vocab, embed_size):
        embeddings = np.zeros((len(vocab), embed_size))
        for word, idx in vocab.items():
            # Encode single token
            embed = embed_model.encode([word])[0]
            embeddings[idx] = embed[:embed_size]  # Truncate to embed_size
        return torch.tensor(embeddings, dtype=torch.float32)

    def forward(self, images, captions, teacher_forcing_ratio=0.5):
        features = self.encoder(images)
        outputs = self.decoder(features, captions, teacher_forcing_ratio)
        return outputs

    @torch.no_grad()
    def generate_caption(self, image, vocab, idx_to_word, max_length=20, beam_width=5):
        self.eval()
        image = image.unsqueeze(0).to(image.device)
        features = self.encoder(image).unsqueeze(1)
        sequences = [[[], 0.0, [features, None]]]
        for _ in range(max_length):
            all_candidates = []
            for seq, score, state in sequences:
                if len(seq) > 0 and seq[-1] == vocab["</s>"]:
                    all_candidates.append([seq, score, state])
                    continue
                if len(seq) == 0:
                    token = torch.tensor([[vocab["<s>"]]], device=image.device)
                else:
                    token = torch.tensor([[seq[-1]]], device=image.device)
                embed = self.decoder.embedding(token)
                context, _ = self.decoder.attention(state[0], state[1][0] if state[1] else torch.zeros(1, self.decoder.hidden_size).to(image.device))
                input_lstm = torch.cat([embed.squeeze(1), context], dim=1)
                if state[1] is None:
                    h, c = self.decoder.lstm(input_lstm)
                else:
                    h, c = self.decoder.lstm(input_lstm, state[1])
                output = self.decoder.fc(h)
                output = torch.softmax(output, dim=-1)
                top_probs, top_indices = output.topk(beam_width)
                for i in range(beam_width):
                    next_seq = seq + [top_indices[0, i].item()]
                    next_score = score + torch.log(top_probs[0, i]).item() - 0.1 * next_seq.count(top_indices[0, i].item())  # Repetition penalty
                    all_candidates.append([next_seq, next_score, [features, (h, c)]])
            sequences = sorted(all_candidates, key=lambda x: x[1], reverse=True)[:beam_width]
        captions = []
        for seq, score, _ in sequences:
            caption = [idx_to_word.get(idx, "<unk>") for idx in seq if idx not in [vocab["<s>"], vocab["</s>"]]]
            captions.append(" ".join(caption))
        return captions

# Early stopping
class EarlyStopping:
    def __init__(self, patience=5):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False

    def __call__(self, val_loss, model, model_path='best_base_resnet50_lstm.pth'):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
            torch.save(model.state_dict(), model_path)
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

## Evaluation

In [19]:
# Data transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load test datasets
test_us_dataset = ImageCaptionDataset(
    image_dir='/kaggle/input/capydata-ic/data/test/images',
    captions_file='/kaggle/input/capydata-ic/data/test/captions.txt',
    vocab=vocab,
    transform=transform
)
test_us_loader = DataLoader(test_us_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

# Model setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ImageCaptionModel(encoded_size=256, embed_size=256, hidden_size=512, vocab_size=len(vocab)).to(device)
model.load_state_dict(torch.load('/kaggle/input/base-resnet50-lstm/pytorch/default/1/best_base_resnet50_lstm.pth', weights_only=True, map_location=device))
model.eval()

# Initialize PhoBERT for BERTScore
phobert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
from bert_score.utils import model2layers
if "vinai/phobert-base" not in model2layers:
    model2layers["vinai/phobert-base"] = 12

# Initialize scorers
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=False)
cider_scorer = Cider()

# Hàm để lấy n-gram từ một chuỗi
def get_ngrams(text, n):
    tokens = text.split()
    return [' '.join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

# Hàm debug n-gram chung giữa ground truth và hypothesis
def debug_ngrams(gts, hyp, max_n=4):
    for n in range(1, max_n+1):
        gts_ngrams = [get_ngrams(gt, n) for gt in gts]
        hyp_ngrams = get_ngrams(hyp, n)
        common_ngrams = set.intersection(*[set(ngrams) for ngrams in gts_ngrams + [hyp_ngrams]])
        print(f"{n}-gram common: {common_ngrams}")
    
# Evaluation function    
def evaluate_model(model, test_loader, vocab, idx_to_word, dataset_name="Unknown"):
    bleu_scores = {f"BLEU-{i}": [] for i in range(1, 5)}
    rouge_l_scores = []
    bert_p_scores = []
    bert_r_scores = []
    bert_f1_scores = []
    cider_scores = []
    all_gts = {}  # Lưu ground truth cho CIDEr
    all_hyp = {}  # Lưu hypothesis cho CIDEr

    smoothing = SmoothingFunction().method1

    for idx, (images, _, img_names, raw_tokens) in enumerate(tqdm(test_loader, desc=f"Evaluating {dataset_name}")):
        image = images[0].to(device)
        ground_truth = [' '.join(tokens) for tokens in raw_tokens]

        # Generate captions
        generated_captions = model.generate_caption(image, vocab, idx_to_word, max_length=30, beam_width=1)
        generated_caption = generated_captions[0]
        generated_tokens = generated_caption.split()

        # Debug
        print(f"Image: {img_names[0]}")
        print(f"Generated Caption: {generated_caption}")
        print(f"Ground Truth Captions: {ground_truth}")
        if not generated_caption.strip() or not any(gt.strip() for gt in ground_truth):
            print("Warning: Empty caption or ground truth detected!")
        debug_ngrams(ground_truth, generated_caption)

        # Lưu ground truth và hypothesis cho CIDEr
        all_gts[idx] = [gt for gt in ground_truth if gt.strip()]
        all_hyp[idx] = [generated_caption if generated_caption.strip() else ""]

        # BLEU-1 to BLEU-4 with best match
        reference_tokens = [tokens for tokens in raw_tokens]
        best_bleu_scores = {f"BLEU-{i}": 0.0 for i in range(1, 5)}
        for ref in reference_tokens:
            for i in range(1, 5):
                bleu = sentence_bleu(
                    [ref],
                    generated_tokens,
                    weights=tuple(1.0 / i if j < i else 0.0 for j in range(4)),
                    smoothing_function=smoothing
                )
                best_bleu_scores[f"BLEU-{i}"] = max(best_bleu_scores[f"BLEU-{i}"], bleu)
        for i in range(1, 5):
            bleu_scores[f"BLEU-{i}"].append(best_bleu_scores[f"BLEU-{i}"])

        # ROUGE-L with best match
        best_rouge_l = 0.0
        for gt in ground_truth:
            rouge_scores = scorer.score(gt, generated_caption)
            best_rouge_l = max(best_rouge_l, rouge_scores['rougeL'].fmeasure)
        rouge_l_scores.append(best_rouge_l)

        # BERTScore with best match
        P, R, F1 = bert_score(
            [generated_caption],
            ground_truth,
            model_type="vinai/phobert-base",
            lang="vi",
            device=device,
            use_fast_tokenizer=True,
            rescale_with_baseline=True
        )
        best_bert_p = P.max().item()
        best_bert_r = R.max().item()
        best_bert_f1 = F1.max().item()
        bert_p_scores.append(best_bert_p)
        bert_r_scores.append(best_bert_r)
        bert_f1_scores.append(best_bert_f1)

    # Tính CIDEr trên toàn bộ dataset
    try:
        cider_score, cider_scores_per_image = cider_scorer.compute_score(all_gts, all_hyp)
        cider_scores = list(cider_scores_per_image)
        print(f"Overall CIDEr score for {dataset_name}: {cider_score:.4f}")
    except Exception as e:
        print(f"Error computing CIDEr: {e}")
        cider_scores = [0.0] * len(all_gts)

    # Compute average scores
    avg_bleu_scores = {f"BLEU-{i}": np.mean(bleu_scores[f"BLEU-{i}"]) for i in range(1, 5)}
    avg_rouge_l = np.mean(rouge_l_scores)
    avg_bert_p = np.mean(bert_p_scores)
    avg_bert_r = np.mean(bert_r_scores)
    avg_bert_f1 = np.mean(bert_f1_scores)
    avg_cider = np.mean(cider_scores)

    # Print results
    print(f"\nEvaluation Results for {dataset_name}:")
    for i in range(1, 5):
        print(f"Average BLEU-{i}: {avg_bleu_scores[f'BLEU-{i}']:.4f}")
    print(f"Average ROUGE-L: {avg_rouge_l:.4f}")
    print(f"Average BERTScore Precision: {avg_bert_p:.4f}")
    print(f"Average BERTScore Recall: {avg_bert_r:.4f}")
    print(f"Average BERTScore F1: {avg_bert_f1:.4f}")
    print(f"Average CIDEr: {avg_cider:.4f}")

    # Return results
    results = {
        **avg_bleu_scores,
        "ROUGE-L": avg_rouge_l,
        "BERTScore_P": avg_bert_p,
        "BERTScore_R": avg_bert_r,
        "BERTScore_F1": avg_bert_f1,
        "CIDEr": avg_cider
    }
    return results

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
# Run evaluation US
print(f"Evaluating on device: {device}")
results_capydata = evaluate_model(model, test_us_loader, vocab, inv_vocab, dataset_name="capydata-ic")

Evaluating on device: cuda


Evaluating capydata-ic:   0%|          | 0/1395 [00:00<?, ?it/s]

Image: 100000001084.jpg
Generated Caption: vận_động_viên đang chuẩn_bị bắn cung với tư_thế vững_chãi và tự_tin
Ground Truth Captions: ['đường_dây cung_cấp_dương như căng_thẳng vận_động_viên tập_trung vào mục_tiêu xa_xa']
1-gram common: {'vận_động_viên'}
2-gram common: set()
3-gram common: set()
4-gram common: set()


Evaluating capydata-ic:   0%|          | 1/1395 [00:01<33:18,  1.43s/it]

Image: 100000001084.jpg
Generated Caption: vận_động_viên đang chuẩn_bị bắn cung với tư_thế vững_chãi và tự_tin
Ground Truth Captions: ['với trang_phục thể_thao vận_động_viên nhẹ_nhàng điều_chỉnh bó cung trước khi bắn']
1-gram common: {'bắn', 'với', 'cung', 'vận_động_viên'}
2-gram common: set()
3-gram common: set()
4-gram common: set()


...

Image: 100000000245.jpg
Generated Caption: vận_động_viên trong trang_phục thể_thao đang chuẩn_bị phát_bóng với sự tập_trung cao_độ
Ground Truth Captions: ['bối_cảnh thi_đấu sôi_động với không_khí quyết_liệt hai đội đang cạnh_tranh quyết_liệt']
1-gram common: {'đang', 'với'}
2-gram common: set()
3-gram common: set()
4-gram common: set()


Evaluating capydata-ic: 100%|█████████▉| 1394/1395 [21:55<00:00,  1.11it/s]

Image: 100000000245.jpg
Generated Caption: vận_động_viên trong trang_phục thể_thao đang thực_hiện cú phát_bóng mạnh_mẽ
Ground Truth Captions: ['cú bật nhảy thuyết_phục từ vị_trí trung_tâm thể_hiện sức_mạnh và kỹ_thuật tinh_tế']
1-gram common: {'cú'}
2-gram common: set()
3-gram common: set()
4-gram common: set()


Evaluating capydata-ic: 100%|██████████| 1395/1395 [21:56<00:00,  1.06it/s]


Overall CIDEr score for capydata-ic: 0.3452

Evaluation Results for capydata-ic:
Average BLEU-1: 0.1857
Average BLEU-2: 0.0896
Average BLEU-3: 0.0515
Average BLEU-4: 0.0345
Average ROUGE-L: 0.3582
Average BERTScore Precision: 0.4768
Average BERTScore Recall: 0.4681
Average BERTScore F1: 0.4720
Average CIDEr: 0.3452


In [21]:
# Run evaluation UIT
test_uit_dataset = ImageCaptionDataset(
    image_dir='/kaggle/input/capydata-caption/data/UIT-ViIC/dataset/test/images',
    captions_file='/kaggle/input/capydata-caption/data/UIT-ViIC/dataset/test/captions.txt',
    vocab=vocab,
    transform=transform
)
test_uit_loader = DataLoader(test_uit_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

print(f"Evaluating on device: {device}")
results_uit_viic = evaluate_model(model, test_uit_loader, vocab, inv_vocab, dataset_name="test_uit_viic")

Evaluating on device: cuda


Evaluating test_uit_viic:   0%|          | 0/2895 [00:00<?, ?it/s]

Image: 000000135082.jpg
Generated Caption: vận_động_viên trong trang_phục thể_thao thoải_mái tập_trung vào cú đánh quyết_định
Ground Truth Captions: ['nhiều người đang luyện_tập bóng_chày ở trong công_viên']
1-gram common: {'trong'}
2-gram common: set()
3-gram common: set()
4-gram common: set()


Evaluating test_uit_viic:   0%|          | 1/2895 [00:00<43:17,  1.11it/s]

Image: 000000135082.jpg
Generated Caption: vận_động_viên trong trang_phục thể_thao thoải_mái tập_trung vào cú đánh quyết_định
Ground Truth Captions: ['một nhóm người đang chơi bóng_chày ở trên sân']
1-gram common: set()
2-gram common: set()
3-gram common: set()
4-gram common: set()


...

Image: 000000401369.jpg
Generated Caption: các vận_động_viên đang tăng_tốc trên đường_phố thể_hiện sự quyết_tâm và quyết_tâm
Ground Truth Captions: ['một người đàn_ông đang hướng_dẫn cách cầm vợt cho một cô bé']
1-gram common: {'đang'}
2-gram common: set()
3-gram common: set()
4-gram common: set()


Evaluating test_uit_viic: 100%|█████████▉| 2894/2895 [44:17<00:00,  1.10it/s]

Image: 000000401369.jpg
Generated Caption: các vận_động_viên đang tăng_tốc trên đường chạy thể_hiện sự quyết_tâm và quyết_tâm
Ground Truth Captions: ['một bé gái đang học cách chơi tennis với người đàn_ông']
1-gram common: {'đang'}
2-gram common: set()
3-gram common: set()
4-gram common: set()


Evaluating test_uit_viic: 100%|██████████| 2895/2895 [44:18<00:00,  1.09it/s]


Overall CIDEr score for test_uit_viic: 0.1224

Evaluation Results for test_uit_viic:
Average BLEU-1: 0.1827
Average BLEU-2: 0.0699
Average BLEU-3: 0.0368
Average BLEU-4: 0.0264
Average ROUGE-L: 0.3235
Average BERTScore Precision: 0.4465
Average BERTScore Recall: 0.4744
Average BERTScore F1: 0.4595
Average CIDEr: 0.1224


In [22]:
# Compare results
print("Comparison of Results:")
print(f"{'Metric':<20} {'capydata-ic':<15} {'test_uit_viic':<15}")
for metric in results_capydata:
    print(f"{metric:<20} {results_capydata[metric]:<15.4f} {results_uit_viic[metric]:<15.4f}")

Comparison of Results:
Metric               capydata-ic     test_uit_viic  
BLEU-1               0.1857          0.1827         
BLEU-2               0.0896          0.0699         
BLEU-3               0.0515          0.0368         
BLEU-4               0.0345          0.0264         
ROUGE-L              0.3582          0.3235         
BERTScore_P          0.4768          0.4465         
BERTScore_R          0.4681          0.4744         
BERTScore_F1         0.4720          0.4595         
CIDEr                0.3452          0.1224         
